#### Objective of this kernel:
* to further explore the data
* To clean the out put the Dataset to be fed into the various machine algorithms

In [8]:
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns


# Input data files are available in the "../output/" directory.
import os
for dirname, _, filenames in os.walk('./output/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# Any results you write to the current directory are saved as output.

./output/linearmodel.pickle
./output/processed_data.csv
./output/cleaned_data.csv
./output/.ipynb_checkpoints/cleaned_data-checkpoint.csv


## Reading the data set

In [2]:
df = pd.read_csv('./output/cleaned_data.csv')
df.shape

(238, 8)

## we are going to transform all our categorical data to proper format to be fed to our machine learning algorithms
* We preprocess our data with the LabelEncoder from sklearn

#### Preprocessing our independent variable x

In [3]:
y= df[["current CGPA"]].values
x =df[["Age","Gender","wassce grade","level","access to a laptop or internet","study group","time spent on independent studies"]].values

In [13]:
from sklearn.preprocessing import LabelEncoder
label_encoder_x = LabelEncoder()
x[:,0] =label_encoder_x.fit_transform(x[:,0])
x[:,1] =label_encoder_x.fit_transform(x[:,1])
x[:,2] =label_encoder_x.fit_transform(x[:,2])
x[:,3] =label_encoder_x.fit_transform(x[:,3])
x[:,4] =label_encoder_x.fit_transform(x[:,4])
x[:,5] =label_encoder_x.fit_transform(x[:,5])
x[:,6] =label_encoder_x.fit_transform(x[:,6])
x.dtype
x=x.astype('int')
x.dtype

dtype('int64')

#### Preocessing our dependent variable y

In [5]:
y[:,0] =label_encoder_x.fit_transform(y[:,0])
y.dtype
y=x.astype('int')
y.dtype

dtype('int64')

# Lets start implementing our machine learning algorithms

In [6]:
from sklearn.model_selection import train_test_split
import pickle
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.4)

# x_train

In [7]:
# importing KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier